# **Meses de oferta (Month of Supply)**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre

    #if nombre.startswith("df_") and "puebla" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo',
 'df_financiamientos_2024_tulancingo']

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Estacionamientos':'estacionamiento'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [2]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['año','mes']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [3]:
# Concatenar los DataFrames
fin_pachuca = pd.concat(dataframes_list)

# Filtrar registros con año diferente de 0
fin_pachuca_filtrado = fin_pachuca[fin_pachuca['año'] != 0]

# Contar los registros por año y mes
conteo_registros_por_mes = fin_pachuca_filtrado.groupby(['año', 'mes']).size().reset_index(name='conteo')
conteo_registros_por_mes

,año,mes,conteo
0,2019,1,16
1,2019,2,28
2,2019,3,29
3,2019,4,33
4,2019,5,40
...,...,...,...
62,2024,3,17
63,2024,4,30
64,2024,5,30
65,2024,6,33


In [6]:
conteos_por_año = fin_pachuca.groupby('año').size()

# Filtrar el conteo para cada año específico
conteo_2022 = conteos_por_año.get(2022, 0)  # Si no hay registros para el año 2022, devolverá 0
conteo_2023 = conteos_por_año.get(2023, 0)  # Si no hay registros para el año 2023, devolverá 0

print("Conteo de registros para 2022:", conteo_2022)
print("Conteo de registros para 2023:", conteo_2023)

Conteo de registros para 2022: 316
Conteo de registros para 2023: 45


In [7]:
promedio_2022 = (conteo_2022/12).round(2)
promedio_2023 = (conteo_2023/12).round(2)
print("Promedio de créditos en 2022:", promedio_2022)
print("Promedio de créditos en 2023:", promedio_2023)

Promedio de créditos en 2022: 26.33
Promedio de créditos en 2023: 3.75


In [8]:
# Dividir los conteos por los promedios respectivos de cada año
conteo_registros_por_mes['promedio_creditos'] = conteo_registros_por_mes.apply(lambda row: round(row['conteo'] / promedio_2022, 2) if row['año'] == 2022 else round(row['conteo'] / promedio_2023, 2), axis=1)
conteo_registros_por_mes

,año,mes,conteo,promedio_creditos
0,2019,1,16,4.27
1,2019,2,28,7.47
2,2019,3,29,7.73
3,2019,4,33,8.80
4,2019,5,40,10.67
5,2019,6,36,9.60
6,2019,7,35,9.33
7,2019,8,40,10.67
8,2019,9,32,8.53
9,2019,10,43,11.47


In [9]:
# Convertir los valores de la columna mes a enteros
conteo_registros_por_mes['mes'] = conteo_registros_por_mes['mes'].astype(int)
# Mapear los números de mes a sus nombres correspondientes
meses_texto = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
# Convertir el número de mes en texto
conteo_registros_por_mes['mes'] = conteo_registros_por_mes.apply(lambda row: f"{meses_texto[row['mes']]}_{int(row['año'])}", axis=1)
# Eliminar la columna 'año' ahora que ya no es necesaria
conteo_registros_por_mes = conteo_registros_por_mes.drop(columns=['año'])
conteo_registros_por_mes

,mes,conteo,promedio_creditos
0,Enero_2019,16,4.27
1,Febrero_2019,28,7.47
2,Marzo_2019,29,7.73
3,Abril_2019,33,8.80
4,Mayo_2019,40,10.67
5,Junio_2019,36,9.60
6,Julio_2019,35,9.33
7,Agosto_2019,40,10.67
8,Septiembre_2019,32,8.53
9,Octubre_2019,43,11.47


In [10]:
# Filtrar solo los registros correspondientes a 2023
conteo_2023 = conteo_registros_por_mes[conteo_registros_por_mes['mes'].str.contains('2023')]
# Seleccionar las columnas mes, conteo y promedio_creditos
conteo_2023 = conteo_2023[['mes', 'conteo', 'promedio_creditos']]
conteo_2023

,mes,conteo,promedio_creditos
48,Enero_2023,4,1.07
49,Febrero_2023,6,1.60
50,Marzo_2023,8,2.13
51,Abril_2023,2,0.53
52,Mayo_2023,5,1.33
53,Junio_2023,6,1.60
54,Julio_2023,3,0.80
55,Agosto_2023,4,1.07
56,Septiembre_2023,7,1.87


In [12]:
# Obtener los datos
promedio = conteo_2023['promedio_creditos']
meses = conteo_2023['mes']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, promedio):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    #title='Meses de oferta',
    xaxis_title='Mes de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)   
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_MonthSupply', carpeta='assets/graficas')
fig.show()